In [31]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [26]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY "))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [28]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Johnson is an amazing professor! Her lectures are clear, and she is always available for extra help.'},
 {'professor': 'Dr. Robert Smith',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great teacher, but his exams are tough. Make sure to attend all classes and do the assignments.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Dr. Davis knows her stuff, but her lectures can be a bit dry. The course material is interesting though.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Challenging course, but Dr. Brown makes the concepts understandable and engaging. Highly recommend.'},
 {'professor': 'Dr. Linda Wilson',
  'subject': 'Chemistry',
  'stars': 4,
  'review': 'Dr. Wilson is a good professor, but her grading is very strict. Pay attention to lab work.'},
 {'professor': 'Dr. James Taylor',
  's

In [32]:
processed_data = []
client = OpenAI()
for review in data['reviews']:
    respons = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = respons.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "meatadata":{
            "reviews": review['review'],
            "subject": review['subject'],
            "stars":review['stars']


        }

    })

In [34]:
processed_data[0]

{'values': [-0.0074431235,
  -0.014648448,
  0.023851287,
  0.04002165,
  0.00029724932,
  -0.006182786,
  0.020831233,
  0.03947471,
  -0.018239219,
  0.0003500111,
  0.017751731,
  -0.0052018636,
  -0.045847736,
  0.025729902,
  -0.018679148,
  0.055787757,
  -0.03124685,
  0.008590505,
  0.016277373,
  0.069437444,
  0.025515882,
  -0.014327418,
  0.026348181,
  -0.030057853,
  -0.03143709,
  -0.027941437,
  0.0064384206,
  -0.00140376,
  0.038024135,
  -0.011016061,
  0.07181544,
  0.0071220943,
  -0.023696717,
  -0.022365041,
  -0.03528944,
  0.023042768,
  0.008138686,
  0.011675954,
  -0.002428527,
  0.010623692,
  -0.017204791,
  0.008477551,
  -0.008073292,
  0.010605857,
  0.037191838,
  -0.0044052354,
  0.00449441,
  -0.00673567,
  0.04858243,
  0.048439752,
  -0.024219876,
  -0.001915772,
  0.020212956,
  -0.0043101152,
  -0.03212671,
  -0.0028848047,
  0.0221748,
  0.028821295,
  0.004170408,
  -0.033553503,
  0.033838864,
  0.011664064,
  -0.021556523,
  -0.02281686,
  -0

In [ ]:
index = pc.index("rag")
index.upsert(
    vectors=processed_data,
    namespace